# Processing the data





Install the Transformers and Datasets libraries to run this notebook.

In [ ]:
! pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 264 kB 7.9 MB/s 
     |████████████████████████████████| 2.6 MB 62.4 MB/s 
     |████████████████████████████████| 118 kB 69.0 MB/s 
     |████████████████████████████████| 243 kB 68.6 MB/s 
     |████████████████████████████████| 76 kB 5.7 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 636 kB 52.8 MB/s 
     |████████████████████████████████| 895 kB 44.3 MB/s 
     |████████████████████████████████| 3.3 MB 56.5 MB/s 
     |████████████████████████████████| 1.1 MB 54.8 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ***EXPLORING THE DATASET***

In [ ]:
import pandas as pd
from transformers import DataCollatorWithPadding
from datasets import load_dataset
from datasets import load_metric

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/NLP -Hugging Face/emotions/train.txt', names=['text', 'emotion'], sep=';')
val_data = pd.read_csv('/content/drive/MyDrive/NLP -Hugging Face/emotions/val.txt', names=['text', 'emotion'], sep=';')
test_data = pd.read_csv('/content/drive/MyDrive/NLP -Hugging Face/emotions/test.txt', names=['text', 'emotion'], sep=';')
train_data.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [ ]:
test_data.head()

,text,emotion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


TRAIN DATA

In [ ]:
train_data['text_len'] = train_data['text'].str.split().str.len()
#adding it to the dataframe
train_data.describe()

,text_len
count,16000.000000
mean,19.166313
std,10.986905
min,2.000000
25%,11.000000
50%,17.000000
75%,25.000000
max,66.000000


Count is number of sentences
max is max count of words in a sentence

In [ ]:
train_data['text']

0                                  i didnt feel humiliated
1        i can go from feeling so hopeless to so damned...
2         im grabbing a minute to post i feel greedy wrong
3        i am ever feeling nostalgic about the fireplac...
4                                     i am feeling grouchy
                               ...                        
15995    i just had a very brief time in the beanbag an...
15996    i am now turning and i feel pathetic that i am...
15997                       i feel strong and good overall
15998    i feel like this was such a rude comment and i...
15999    i know a lot but i feel so stupid because i ca...
Name: text, Length: 16000, dtype: object

In [ ]:
train_data['text'].str.split()

0                             [i, didnt, feel, humiliated]
1        [i, can, go, from, feeling, so, hopeless, to, ...
2        [im, grabbing, a, minute, to, post, i, feel, g...
3        [i, am, ever, feeling, nostalgic, about, the, ...
4                                [i, am, feeling, grouchy]
                               ...                        
15995    [i, just, had, a, very, brief, time, in, the, ...
15996    [i, am, now, turning, and, i, feel, pathetic, ...
15997                [i, feel, strong, and, good, overall]
15998    [i, feel, like, this, was, such, a, rude, comm...
15999    [i, know, a, lot, but, i, feel, so, stupid, be...
Name: text, Length: 16000, dtype: object

In [ ]:
train_data['emotion'].value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: emotion, dtype: int64

In [ ]:
labels = train_data['emotion'].unique().tolist() # list of all 6 emotions
labels_dict = {k: v for v, k in enumerate(labels)} #make s dictionary out of them
labels_dict

{'anger': 1, 'fear': 4, 'joy': 5, 'love': 2, 'sadness': 0, 'surprise': 3}

In [ ]:
train_data['labels'] = train_data['emotion'].map(labels_dict)
train_data.head()

,text,emotion,text_len,labels
0,i didnt feel humiliated,sadness,4,0
1,i can go from feeling so hopeless to so damned...,sadness,21,0
2,im grabbing a minute to post i feel greedy wrong,anger,10,1
3,i am ever feeling nostalgic about the fireplac...,love,18,2
4,i am feeling grouchy,anger,4,1


## ***VALIDATION DATASET***

In [ ]:
val_data['emotion'].value_counts()

joy         704
sadness     550
anger       275
fear        212
love        178
surprise     81
Name: emotion, dtype: int64

In [ ]:
val_data['text_len'] = val_data['text'].str.split().str.len()
val_data['labels'] = val_data['emotion'].map(labels_dict)
val_data.head()

,text,emotion,text_len,labels
0,im feeling quite sad and sorry for myself but ...,sadness,15,0
1,i feel like i am still looking at a blank canv...,sadness,15,0
2,i feel like a faithful servant,love,6,2
3,i am just feeling cranky and blue,anger,7,1
4,i can have for a treat or if i am feeling festive,joy,12,5


# ***TOKENIZING***

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)
train_dataset, train_dataset[0]

(Dataset({
     features: ['text', 'emotion', 'text_len', 'labels'],
     num_rows: 16000
 }),
 {'emotion': 'sadness',
  'labels': 0,
  'text': 'i didnt feel humiliated',
  'text_len': 4})

In [ ]:
from datasets import Dataset
val_dataset = Dataset.from_pandas(val_data)
val_dataset, val_dataset[0]

(Dataset({
     features: ['text', 'emotion', 'text_len', 'labels'],
     num_rows: 2000
 }),
 {'emotion': 'sadness',
  'labels': 0,
  'text': 'im feeling quite sad and sorry for myself but ill snap out of it soon',
  'text_len': 15})

In [ ]:
from transformers import AutoTokenizer
from transformers import BertTokenizer, BertModel
model_checkpoint = 'bert-base-uncased'
tokenizer =  BertTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_dataset

Dataset({
    features: ['text', 'emotion', 'text_len', 'labels'],
    num_rows: 16000
})

In [ ]:
import torch
max_len = 100 #since subwords used 
#ASK how do u know
pad_to_max = True
def tokenize_data(example):
    # Tokenize the review body
    text_ = example['text']
    encodings = tokenizer.encode_plus(text_, padding=True, max_length=max_len,
                                            truncation=True,
                                           add_special_tokens=True,
                                            return_token_type_ids=False,
                                            return_attention_mask=True,
                                            return_overflowing_tokens=False,
                                            return_special_tokens_mask=False,
                                           )
    
    # Subtract 1 from labels to have them in range 0-4
    targets = torch.tensor(example['labels'],dtype=torch.long)
    

    encodings.update({'labels': targets})
    return encodings

In [ ]:
encoded_train_dataset = train_dataset.map(tokenize_data)
encoded_val_dataset = val_dataset.map(tokenize_data)

#ask

  0%|          | 0/16000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

In [ ]:
encoded_train_dataset.column_names

['text', 'emotion', 'text_len', 'labels', 'input_ids', 'attention_mask']

Two more - 'input_ids', 'attention_mask'added

In [ ]:
encoded_train_dataset[:5]

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1]],
 'emotion': ['sadness', 'sadness', 'anger', 'love', 'anger'],
 'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102],
  [101,
   1045,
   2064,
   2175,
   2013,
   3110,
   2061,
   20625,
   2000,
   2061,
   9636,
   17772,
   2074,
   2013,
   2108,
   2105,
   2619,
   2040,
   14977,
   1998,
   2003,
   8300,
   102],
  [101, 10047, 9775, 1037, 3371, 2000, 2695, 1045, 2514, 20505, 3308, 102],
  [101,
   1045,
   2572,
   2412,
   3110,
   16839,
   9080,
   12863,
   2055,
   1996,
   13788,
   1045,
   2097,
   2113,
   2008,
   2009,
   2003,
   2145,
   2006,
   1996,
   3200,
   102],
  [101, 1045, 2572, 3110, 24665, 7140, 11714, 102]],
 'labels': [0, 0, 1, 2, 1],
 'text': ['i didnt feel humiliated',
  'i can go from feeli

# ***LOADING THE MODEL***

https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import BertTokenizer, BertModel
batch_size = 128
num_labels = 6


model_checkpoint = 'bert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# ***TRAINER***

In [ ]:
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
print(metrics_list)

['accuracy', 'bertscore', 'bleu', 'bleurt', 'cer', 'comet', 'coval', 'cuad', 'f1', 'gleu', 'glue', 'indic_glue', 'matthews_correlation', 'meteor', 'pearsonr', 'precision', 'recall', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'wer', 'wiki_split', 'xnli']


In [ ]:
acc_metric = load_metric('accuracy')
f1_metric = load_metric('f1')
precision_metric = load_metric('precision')
recall_metric = load_metric('recall')

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

In [ ]:
metric_name = "accuracy" #why only accuracy

args = TrainingArguments(
    output_dir = "test-results-concat",
    seed = 125, 
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    eval_steps = 100,
    save_steps = 100,
    fp16 = False
)

In [ ]:
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')
    print(acc, f1,recall, precision)
    return {"accuracy": acc['accuracy'], "f1": f1['f1'],"recall": recall['recall'],"precision": precision['precision']} 

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset= encoded_train_dataset, 
    eval_dataset=encoded_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
!nvidia-smi #ask

Mon Aug  2 20:46:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    34W / 250W |   1381MiB / 16280MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, emotion, text_len.
***** Running training *****
  Num examples = 16000
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 500


Step,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
100,No log,0.696278,0.777500,0.739774,0.777500,0.748841
200,No log,0.305296,0.915500,0.914515,0.915500,0.916502
300,No log,0.220269,0.923500,0.922625,0.923500,0.923064
400,No log,0.190398,0.931500,0.931603,0.931500,0.931798
500,0.454000,0.184278,0.929500,0.929626,0.929500,0.930174


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, emotion, text_len.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 128


{'accuracy': 0.7775} {'f1': 0.7397740444589191} {'recall': 0.7775} {'precision': 0.748840530772581}


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to test-results-concat/checkpoint-100
Configuration saved in test-results-concat/checkpoint-100/config.json
Model weights saved in test-results-concat/checkpoint-100/pytorch_model.bin
tokenizer config file saved in test-results-concat/checkpoint-100/tokenizer_config.json
Special tokens file saved in test-results-concat/checkpoint-100/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, emotion, text_len.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 128


...

{'accuracy': 0.9315} {'f1': 0.9316028474546958} {'recall': 0.9315} {'precision': 0.9317980543138953}


Saving model checkpoint to test-results-concat/checkpoint-400
Configuration saved in test-results-concat/checkpoint-400/config.json
Model weights saved in test-results-concat/checkpoint-400/pytorch_model.bin
tokenizer config file saved in test-results-concat/checkpoint-400/tokenizer_config.json
Special tokens file saved in test-results-concat/checkpoint-400/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, emotion, text_len.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 128


{'accuracy': 0.9295} {'f1': 0.9296257809744644} {'recall': 0.9295} {'precision': 0.9301744223005919}


Saving model checkpoint to test-results-concat/checkpoint-500
Configuration saved in test-results-concat/checkpoint-500/config.json
Model weights saved in test-results-concat/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-results-concat/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-results-concat/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from test-results-concat/checkpoint-400 (score: 0.9315).


TrainOutput(global_step=500, training_loss=0.4539955749511719, metrics={'train_runtime': 385.1807, 'train_samples_per_second': 166.156, 'train_steps_per_second': 1.298, 'total_flos': 1988598995394048.0, 'train_loss': 0.4539955749511719, 'epoch': 4.0})

In [ ]:
import numpy as np
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, emotion, text_len.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 128


{'accuracy': 0.9315} {'f1': 0.9316028474546958} {'recall': 0.9315} {'precision': 0.9317980543138953}


{'epoch': 4.0,
 'eval_accuracy': 0.9315,
 'eval_f1': 0.9316028474546958,
 'eval_loss': 0.1903982311487198,
 'eval_precision': 0.9317980543138953,
 'eval_recall': 0.9315,
 'eval_runtime': 3.6701,
 'eval_samples_per_second': 544.948,
 'eval_steps_per_second': 4.36}

In [ ]:
trainer.save_model('Bert_classification_model')

Saving model checkpoint to Bert_classification_model
Configuration saved in Bert_classification_model/config.json
Model weights saved in Bert_classification_model/pytorch_model.bin
tokenizer config file saved in Bert_classification_model/tokenizer_config.json
Special tokens file saved in Bert_classification_model/special_tokens_map.json


In [ ]:
!zip -r 'Bert_classification_model.zip' 'Bert_classification_model'


  adding: Bert_classification_model/ (stored 0%)
  adding: Bert_classification_model/config.json (deflated 53%)
  adding: Bert_classification_model/special_tokens_map.json (deflated 40%)
  adding: Bert_classification_model/pytorch_model.bin (deflated 7%)
  adding: Bert_classification_model/vocab.txt (deflated 53%)
  adding: Bert_classification_model/tokenizer_config.json (deflated 40%)
  adding: Bert_classification_model/training_args.bin (deflated 46%)


In [ ]:
!mv 'Bert_classification_model.zip' 
#old location, new location

mv: missing destination file operand after 'Bert_classification_model.zip'
Try 'mv --help' for more information.


# ***TEST***

In [ ]:
test_data['text_len'] = test_data['text'].str.split().str.len()
test_data['labels'] = test_data['emotion'].map(labels_dict)
test_data.head()

,text,emotion,text_len,labels
0,im feeling rather rotten so im not very ambiti...,sadness,11,0
1,im updating my blog because i feel shitty,sadness,8,0
2,i never make her separate from me because i do...,sadness,22,0
3,i left with my bouquet of red and yellow tulip...,joy,21,5
4,i was feeling a little vain when i did this one,sadness,11,0


In [ ]:
from datasets import Dataset
test_dataset = Dataset.from_pandas(test_data)
test_dataset, test_dataset[0]

(Dataset({
     features: ['text', 'emotion', 'text_len', 'labels'],
     num_rows: 2000
 }),
 {'emotion': 'sadness',
  'labels': 0,
  'text': 'im feeling rather rotten so im not very ambitious right now',
  'text_len': 11})

In [ ]:
encoded_test_dataset = test_dataset.map(tokenize_data)

  0%|          | 0/2000 [00:00<?, ?ex/s]

In [ ]:
encoded_test_dataset.column_names

['text', 'emotion', 'text_len', 'labels', 'input_ids', 'attention_mask']

In [ ]:
test_predictions = trainer.predict(encoded_test_dataset )

test_preds = np.argmax(test_predictions.predictions, axis=-1)

acc_metric = load_metric('accuracy')
f1_metric = load_metric('f1')
precision_metric = load_metric('precision')
recall_metric = load_metric('recall')

acc_metric.compute(predictions=test_preds, references=test_predictions.label_ids)
f1_metric.compute(predictions=test_preds, references=test_predictions.label_ids, average='weighted')
precision_metric .compute(predictions=test_preds, references=test_predictions.label_ids,average='weighted')
recall_metric.compute(predictions=test_preds, references=test_predictions.label_ids,average='weighted')

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, emotion, text_len.
***** Running Prediction *****
  Num examples = 2000
  Batch size = 128


{'accuracy': 0.928} {'f1': 0.9279742865810602} {'recall': 0.928} {'precision': 0.9283996015544033}


{'recall': 0.928}

OTHER METHOD

In [ ]:
trainer = Trainer(
    model,
    args,

    eval_dataset=encoded_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, emotion, text_len.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 128


{'accuracy': 0.928} {'f1': 0.9279742865810602} {'recall': 0.928} {'precision': 0.9283996015544033}


{'eval_accuracy': 0.928,
 'eval_f1': 0.9279742865810602,
 'eval_loss': 0.18621191382408142,
 'eval_precision': 0.9283996015544033,
 'eval_recall': 0.928,
 'eval_runtime': 3.6088,
 'eval_samples_per_second': 554.206,
 'eval_steps_per_second': 4.434}

The test dataset performance is poor as compared to validation dataset